In [6]:
import {InferenceSession, Tensor} from 'onnxruntime-node';
const ndarray = require('ndarray')
const ops = require('ndarray-ops')
const fs = require('fs')
const bisweb = require('biswebnode')



In [ ]:
import {InferenceSession, Tensor} from 'onnxruntime-node';
const ndarray = require('ndarray')
const ops = require('ndarray-ops')
const fs = require('fs')
const bisweb = require('biswebnode')


function imageDataToTensor(data, dims){
  // 1a. Extract the R, G, and B channels from the data to form a 3D int array
  const [R, G, B] = new Array([], [], []);
  for (let i = 0; i < data.length; i += 4) {
    R.push(data[i]);
    G.push(data[i + 1]);
    B.push(data[i + 2]);
    // 2. skip data[i + 3] thus filtering out the alpha channel
  }
  ///console.log(R);
  //console.log(G);
  //console.log(B);
  // 1b. concatenate RGB ~= transpose [224, 224, 3] -> [3, 224, 224]
  const transposedData = R.concat(G).concat(B);

  // 3. convert to float32
  let i, l = transposedData.length; // length, we need this for the loop
  const float32Data = new Float32Array(dims[0]*dims[1]*dims[2]); // create the Float32Array for output
  for (i = 0; i < l; i++) {
    float32Data[i] = transposedData[i] ; // convert to float
  }

  const inputTensor = new Tensor("float32", float32Data, dims);
  return inputTensor;
}

let img=new bisweb.BisWebImage();
var data;


// use an async context to call onnxruntime functions.
async function main() {
    try {
        // create a new session and load the specific model.
        //
        // the model in this example contains a single MatMul node
        // it has 2 inputs: 'a'(float32, 3x4) and 'b'(float32, 4x3)
        // it has 1 output: 'c'(float32, 3x3)
        img.load("../model/head.nii.gz").then( () => {
          // console.log('Image Loaded = ',img.getImageData(), img.internal.dimensions);
          return img.getImageData()
        }).catch( (e) => {
          console.log(e,e.stack);
        });
        data = imageDataToTensor(img, [211,224,224,1])
        console.log(data.type)

        const session = await InferenceSession.create('../model/model.onnx');
        console.log(session.inputNames[0], data)

        // prepare feeds. use model input names as keys.
        const feeds = { input_1:data };

        // feed inputs and run
        const results = await session.run(feeds);

        // read from results
        const dataC = results.c.data;
        console.log(`data of result tensor 'c': ${dataC}`);

    } catch (e) {
        console.error(`failed to inference ONNX model: ${e}.`);
    }
}

main();

float32
Promise { <pending> }


input_1 l {
  dims: [ 211, 224, 224, 1 ],
  type: 'float32',
  data: Float32Array(10587136) [
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0,
    ... 10587036 more items
  ],
  size: 10587136
}


failed to inference ONNX model: TypeError: Cannot read property 'data' of undefined.


++++	 loaded image from ../model/head.nii.gz. Dim= [ 204, 224, 160, 1, 1 ] RAS spa=1,1,1,0.02500000037252903,1 type=sshort


In [7]:
function imageDataToTensor(data, dims){
  // 1a. Extract the R, G, and B channels from the data to form a 3D int array
  const [R, G, B] = new Array([], [], []);
  for (let i = 0; i < data.length; i += 4) {
    R.push(data[i]);
    G.push(data[i + 1]);
    B.push(data[i + 2]);
    // 2. skip data[i + 3] thus filtering out the alpha channel
  }
  ///console.log(R);
  //console.log(G);
  //console.log(B);
  // 1b. concatenate RGB ~= transpose [224, 224, 3] -> [3, 224, 224]
  const transposedData = R.concat(G).concat(B);

  // 3. convert to float32
  let i, l = transposedData.length; // length, we need this for the loop
  const float32Data = new Float32Array(dims[0]*dims[1]*dims[2]); // create the Float32Array for output
  for (i = 0; i < l; i++) {
    float32Data[i] = transposedData[i] / 255.0; // convert to float
  }

  const inputTensor = new Tensor("float32", float32Data, dims);
  return inputTensor;
}

In [10]:
// create an inference session, using WebGL backend. (default is 'wasm') 
//const session = await ort.InferenceSession.create('./model/squeezenet1_1.onnx', { executionProviders: ['wasm'] }); 
const session = await InferenceSession.create('../model/model.onnx', { executionProviders: ['wasm'] });

TypeError: Only absolute URLs are supported
    at getNodeRequestOptions (/home/thuy/repo/deepsyence/node_modules/node-fetch/lib/index.js:1305:9)
    at /home/thuy/repo/deepsyence/node_modules/node-fetch/lib/index.js:1410:19
    at new Promise (<anonymous>)
    at fetch (/home/thuy/repo/deepsyence/node_modules/node-fetch/lib/index.js:1407:9)
    at Object.createSessionHandler (/home/thuy/repo/deepsyence/node_modules/onnxruntime-web/dist/ort-web.node.js:6:159912)
    at Function.create (/home/thuy/repo/deepsyence/node_modules/onnxruntime-common/dist/ort-common.node.js:6:6607)
    at async evalmachine.<anonymous>:5:17
    at async Object.execute (/home/thuy/.nvm/versions/node/v14.4.0/lib/node_modules/tslab/dist/executor.js:175:17)
    at async JupyterHandlerImpl.handleExecuteImpl (/home/thuy/.nvm/versions/node/v14.4.0/lib/node_modules/tslab/dist/jupyter.js:219:18)


In [74]:


async function runModel(model, preprocessedData) {
  const start = new Date();
  try {
    const input = preprocessedData;
    const feeds = {float_input: input};
    // feeds[model.inputNames[0]] = preprocessedData;
    const outputData = await model.run(feeds);
    const end = new Date();
    const inferenceTime = (end.getTime() - start.getTime());
    const output = outputData[model.outputNames[0]];
    return [output, inferenceTime];
  } catch (e) {
    console.error(e);
    throw new Error();
  }
}

//The softmax transforms values to be between 0 and 1
function softmax(resultArray) {
  // Get the largest value in the array.
  const largestNumber = Math.max(...resultArray);
  // Apply exponential function to each result item subtracted by the largest number, use reduce to get the previous result number and the current number to sum all the exponentials results.
  const sumOfExp = resultArray.map((resultItem) => Math.exp(resultItem - largestNumber)).reduce((prevNumber, currentNumber) => prevNumber + currentNumber);
  //Normalizes the resultArray by dividing by the sum of all exponentials; this normalization ensures that the sum of the components of the output vector is 1.
  return resultArray.map((resultValue, index) => {
    return Math.exp(resultValue - largestNumber) / sumOfExp;
  });
}

const [res, time] =  await runModel(session, data);
var output = res.data;
var inferenceTime = time;
var results = softmax(Array.prototype.slice.call(output))

// var topResults = [];
// for (let i = 0; i < results.length; i++) {
//   if (results[i] > 0.3) {
//     topResults.push([classes[i] + ": " + results[i]]);
//   }
// }

console.log(inferenceTime);

TypeError: Only absolute URLs are supported
    at getNodeRequestOptions (/home/thuy/repo/deepsyence/node_modules/node-fetch/lib/index.js:1305:9)
    at /home/thuy/repo/deepsyence/node_modules/node-fetch/lib/index.js:1410:19
    at new Promise (<anonymous>)
    at fetch (/home/thuy/repo/deepsyence/node_modules/node-fetch/lib/index.js:1407:9)
    at Object.createSessionHandler (/home/thuy/repo/deepsyence/node_modules/onnxruntime-web/dist/ort-web.node.js:6:159912)
    at Function.create (/home/thuy/repo/deepsyence/node_modules/onnxruntime-common/dist/ort-common.node.js:6:6607)
    at async evalmachine.<anonymous>:5:17
    at async Object.execute (/home/thuy/.nvm/versions/node/v14.4.0/lib/node_modules/tslab/dist/executor.js:175:17)
    at async JupyterHandlerImpl.handleExecuteImpl (/home/thuy/.nvm/versions/node/v14.4.0/lib/node_modules/tslab/dist/jupyter.js:219:18)


In [ ]:

async function runModel(model, preprocessedData) {
    const start = new Date();
    try {
      const input = new ort.Tensor(new Float32Array(preprocessedData), [1, 380]);
      const feeds = {float_input: input};
      // feeds[model.inputNames[0]] = preprocessedData;
      const outputData = await model.run(feeds);
      const end = new Date();
      const inferenceTime = (end.getTime() - start.getTime());
      const output = outputData[model.outputNames[0]];
      return [output, inferenceTime];
    } catch (e) {
      console.error(e);
      throw new Error();
    }
  }

In [ ]:
//The softmax transforms values to be between 0 and 1
function softmax(resultArray) {
  // Get the largest value in the array.
  const largestNumber = Math.max(...resultArray);
  // Apply exponential function to each result item subtracted by the largest number, use reduce to get the previous result number and the current number to sum all the exponentials results.
  const sumOfExp = resultArray.map((resultItem) => Math.exp(resultItem - largestNumber)).reduce((prevNumber, currentNumber) => prevNumber + currentNumber);
  //Normalizes the resultArray by dividing by the sum of all exponentials; this normalization ensures that the sum of the components of the output vector is 1.
  return resultArray.map((resultValue, index) => {
    return Math.exp(resultValue - largestNumber) / sumOfExp;
  });
}

In [75]:
const [res, time] =  await runModel(session, data);
var output = res.data;
var inferenceTime = time;
var results = softmax(Array.prototype.slice.call(output))

1:28 - Cannot find name 'runModel'.
1:37 - Cannot find name 'session'.
4:15 - Cannot find name 'softmax'.


In [ ]:
var topResults = [];
for (let i = 0; i < results.length; i++) {
  if (results[i] > 0.3) {
    topResults.push([classes[i] + ": " + results[i]]);
  }
}

console.log(topResults);